In [1]:
### Env: pipy335
import pandas as pd
import fastai as fastai
import numpy as np
import autogluon as ag
from autogluon import TabularPrediction as task
from sklearn import metrics
from fastai.tabular.transform import Categorify, FillMissing, Normalize, add_datepart
import math

In [2]:
### IMPORT THE FILE WITH DATASET
inpDf = pd.read_csv('/data2/home/prasannaiyer/Projects/TT_AG/Dataset/OTD_2019_PU.csv')
inpDf = inpDf[(inpDf['Days Late']<20)&((inpDf['Location Type']=='Plant')\
    |(inpDf['Location Type']=='Port')|(inpDf['Location Type']=='Keen'))]

In [3]:
### RENAME COLUMNS AS WELL AS SET DATATYPE OF DATE FEATURES  ###
inpDf['Delivery_Date'] = pd.to_datetime(inpDf['Shipment End Date'])
inpDf['Pickup_Date'] = pd.to_datetime(inpDf['First shipped date (first YT26 action) Date'])
inpDf.rename(columns = {'Shipment Secure Resources Upd Dt':'Tender_Date'},inplace = True)
inpDf['Tender_Date'] = pd.to_datetime(inpDf['Tender_Date'])

In [4]:
### CREATE NEW FEATURES
### OutputTransitTime IS THE DEPENDENT VARIABLE
### OD IS A FEATURE FOR OVER-DIMENTIONSAL SHIPMENTS
inpDf['OutputTransitTime'] = (inpDf['Delivery_Date']-inpDf['Pickup_Date']).dt.days
inpDf['OD'] = [1 if OD > 0 else 0 for OD in inpDf['Shipment OD Day']]

In [5]:
### REMOVE OUTLIERS BASED ON CRITERIA FROM EDA ###
inpDf = inpDf[(inpDf['OutputTransitTime'] <16) & (inpDf['OutputTransitTime'] > 0)]
inpDf = inpDf[(inpDf['Shipment Loaded Distance']>200) & (inpDf['Shipment Loaded Distance']<2501)]
### DROP NULL VALUES OF DEPENDENT VARIABLE ###
inpDf.dropna(subset=['OutputTransitTime'],inplace=True)

In [6]:
### DROP COLUMNS NOT NEEDED FOR PREDICTION ###
inpDf.drop(['Shipment GID', 'Delivery Status','Shipment Source Location GID',\
    'Shipment Source Province Code','Shipment Destination Location GID',\
        'Shipment Destination Location Name', 'Shipment Destination City',\
            'Shipment Enroute Status Upd Dt','Vehicle Outbound Delivery tendered and accepted Date',\
                'First shipped date (first YT26 action) Date', 'Shipment End Date',\
                    'Expected Days to Deliver', 'Actual Days', 'Days Late','Year',\
                        'Month','Shipment OD Day','Pickup_Date','Delivery_Date'],\
                            inplace = True,axis=1)

In [7]:
### CREATION OF NEW FEATURES FROM EXISTING DATE FEATURES ###
inpDf = add_datepart(inpDf,'Tender_Date',drop=False)

Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [8]:
### SORT THE DATAFRAME ###
### Next step is to split the data into training and testing dataset. 
### Since this data involves dates, dataset is sorted by the specific date feature. 
### This ensures that older data is used for training and later data is used in testing. 
### This is important as the model will be used to predict future values of the dependent variable
inpDf = inpDf.sort_values(by=['Tender_Date'])
inpDf = inpDf.drop(['Tender_Date'],axis=1)
data_count = inpDf.shape[0]
data_train_count = int(data_count*0.8)
data_val_count = data_count - data_train_count
train_data = inpDf[:data_train_count]
test_data = inpDf[data_train_count:]

In [9]:
labelColumn = 'OutputTransitTime'

In [10]:
### Autogluon requires directories for saving the output
modelDir = '/home/prasannaiyer/Projects/AGModel1'
modelDir1 = '/home/prasannaiyer/Projects/AGModel2'

In [11]:
### HYPER PARAMETERS FOR VARIOUS ML ALGORITHMS
nn_options = { 'num_epochs': 10}
#hyperparameters = {'NN': nn_options}
problemType = 'regression'
hyperparameters={'NN': {'num_epochs':20},'GBM': {'num_boost_round':1000},'CAT': {'iterations':10000},'RF': {'n_estimators':300},\
    'XT': {'n_estimators':300},'KNN': {},'custom': ['GBM']}

hp1={'GBM': {'num_boost_round':10000},'CAT': {'iterations':10000},'RF': {'n_estimators':500},\
    'XT': {'n_estimators':500},'KNN': {},'custom': ['GBM']}

time_limits = 2*60 

In [12]:
### OPTION 1: TRAIN ON INDIVIDUAL ML ALGORITHMS
predModel = task.fit(train_data = train_data,label = labelColumn,hyperparameters = hp1,\
    output_directory = modelDir,problem_type = problemType)

### OPTION 1: TRAIN AN ENSEMBLE MODEL
predModel1 = task.fit(train_data = train_data,label = labelColumn,hyperparameters = hp1,\
    output_directory = modelDir1,problem_type = problemType,num_bagging_folds=5,stack_ensemble_levels=1)

### PREDICT THE OUTPUT ON TEST DATA
test_data_nolabel = test_data.drop(labels = [labelColumn],axis=1)

predOutput = predModel.predict(test_data_nolabel)
predOutputEnsemble = predModel1.predict(test_data_nolabel)

### Calculate the root mean squared error
x = math.sqrt(metrics.mean_squared_error(test_data[labelColumn], predOutput))

Beginning AutoGluon training ...
Preprocessing data ...
	Data preprocessing and feature engineering runtime = 0.16s ...
AutoGluon will gauge predictive performance using evaluation metric: root_mean_squared_error
To change this, specify the eval_metric argument of fit()
/data2/prasannaiyer/envs/pipy335/lib/python3.7/imp.py:342: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  return _load(spec)
Fitting model: RandomForestRegressorMSE ...
	3.89s	 = Training runtime
	-2.3592	 = Validation root_mean_squared_error score
Fitting model: ExtraTreesRegressorMSE ...
	2.5s	 = Training runtime
	-2.4441	 = Validation root_mean_squared_error score
Fitting model: KNeighborsRegressorUnif ...
	0.03s	 = Training runtime
	-2.7712	 = Validation root_mean_squared_error score
Fitting model: KNeighborsRegressorDist ...
	0.03s	 = Training runtime
	-2.9917	 = Validation root_mean_squared_error 